# Decorators in Python
As Python developers, we often want to extend the behaviour of a function, like adding logging, timing, caching, or validation, without rewriting its core logic. This is where **decorators** come in. **Decorators** allow us to “wrap” functions with additional functionality in a clean and reusable way.

They are a key part of Python’s functional programming features and are widely used in real-world libraries, especially in frameworks for data science, web development, and automation.

At a basic level, a decorator is a function that takes another function as input and returns a modified version of it. This can help you avoid repetition, organise code better, and make enhancements without altering the function itself. 

In [1]:
# Example to illustrate the use of a decorated function
def greet(func):
    def wrapper():
        print('hello!')
        func()
    return wrapper

@greet
def welcome():
    print('Welcome to this session')

In [2]:
welcome()

hello!
Welcome to this session


The **@greet** line is a decorator, and it applies the **greet()** function to **welcome()**.

Decorators are especially useful when working with logging, authentication, retry logic, or caching. In data workflows, they allow functions to remain clean while still providing helpful add-ons like performance tracking or result caching.

## 1. @lru_cache – Cache Function Results

### What is `@lru_cache`?
In many tasks, especially those involving recursion or repeated computation, we end up calling the same function with the same inputs multiple times. This wastes time and resources.
This decorator from the `functools` module stores the result of expensive function calls.  
If the same inputs are provided again, it returns the cached result — instead of recomputing.

**LRU** stands for **Least Recently Used**. It is a caching strategy that keeps track of recent calls. When the cache reaches its size limit, it removes the least recently used items first. The @lru_cache decorator stores the results of function calls and reuses them when the same inputs appear again. 

#### Business Use Case: Currency Conversion Rates

You run an app that fetches the current exchange rate from an external source (assume it’s slow).  
You want to cache this result for faster repeated access during the same session.


In [3]:
from functools import lru_cache
import time

In [4]:
# Simulated slow exchange rate fetcher

@lru_cache(maxsize=10)  # Cache all unique input calls
def get_exchange_rate(currency: str) -> float:
    
    print(f"Fetching rate for {currency}...")
    
    time.sleep(2)  # Simulate delay
    
    return {
        "USD": 83.12,
        "EUR": 89.45,
        "JPY": 0.56
    }.get(currency, 1.0)

In [5]:
# First call – takes time

print(get_exchange_rate("USD"))

Fetching rate for USD...
83.12


In [7]:
# Second call – instant from cache

print(get_exchange_rate("USD"))

83.12


In [12]:
# Another example
from functools import lru_cache
import time

@lru_cache(maxsize=3)
def slow_add(x,y):
    time.sleep(2)
    return x+y

In [13]:
print(slow_add(2,3)) # this will take 2 seconds

5


In [15]:
print(slow_add(2,3)) # this will return instantly

5


In the first call, the function takes 2 seconds. On the second call with the same inputs, the result is returned instantly from the cache. You can set a **maxsize** (the number of recent calls to store) or use None for unlimited caching. You can also inspect the cache with **.cache_info()** and clear it using **.cache_clear()**.

This can be particularly useful for:
- Recursive functions (like Fibonacci)
- Expensive computations with repeated inputs
- Data lookup or API functions

Using **@lru_cache** can dramatically improve performance in scenarios with repeated function calls and predictable results. It reduces redundant computation and makes your code more efficient without any structural changes.



## 2. @cache – Cache Function Results Without Size Limit

### What is `@cache`?
Starting with Python 3.9, a new decorator named **@cache** was added as a simpler alternative to **@lru_cache**. While **lru_cache** keeps track of which results are used most recently (and discards older ones if the cache is full), **@cache** stores all function results indefinitely. It’s ideal for cases where the number of unique calls is small and doesn't need limiting.

Caches all results of a function **without limiting** the cache size.  
Great for pure functions where you want to avoid recalculations entirely.

#### Business Use Case: Calculating Sales Tax Rates

Suppose you have a function that calculates tax based on location.  
Locations don’t change often, so caching saves redundant calculations.


In [16]:
from functools import cache
import time

In [23]:
# Function to calculate tax based on location and amount
# Simulates an expensive calculation by sleeping for 1 second
# The @cache decorator stores results to avoid recalculating for same inputs

@cache
def calculate_tax(location: str, amount: float) -> float:
    
    print(f"Calculating tax for {location} on amount {amount}")
    time.sleep(1)  # Simulate expensive calculation
    tax_rates = {"NY": 0.08, "CA": 0.075, "TX": 0.065}
    rate = tax_rates.get(location, 0.05)
    
    return amount * rate

In [24]:
# First call — slow

print(calculate_tax("NY", 100))

Calculating tax for NY on amount 100
8.0


In [25]:
# Second call with same params — fast, cached

print(calculate_tax("NY", 100))

8.0


In [26]:
# Different parameters — slow again

print(calculate_tax("CA", 200))

Calculating tax for CA on amount 200
15.0


In [27]:
# Another example
from functools import cache
import time

@cache
def slow_multiply(x,y):
    time.sleep(2)
    return x*y

In [28]:
print(slow_multiply(4,5)) # takes 2 seconds

20


In [30]:
print(slow_multiply(4,5)) # returns instantly

20


Unlike **@lru_cache, @cache** has no **maxsize** and doesn’t remove any values. It keeps all results as long as the program is running. 
This is especially useful when:
- You want a lightweight caching solution
- You don’t need control over memory usage
- The function has a limited and predictable input space

However, it’s not recommended when dealing with a large number of unique inputs, as it can grow unbounded in memory.

**@cache** is a minimal, efficient decorator for speeding up deterministic functions with repeat inputs. It's useful for small, repeated computations where memory limits are not a concern.

## 3. @wraps – Keep Original Function Information

### What is `@wraps`?

When you write your own decorator, the original function’s name, docstring, and other metadata get replaced by the wrapper function.  
`@wraps` from `functools` copies those attributes back to the wrapper so debugging and introspection still work smoothly.

#### Business Use Case: Logging Decorator for API Calls

Imagine you want to add simple logging to your API call functions without losing their original names and documentation.

In [31]:
from functools import wraps

In [32]:
# Our custom decorator to log function calls

def log_calls(func):
    
    @wraps(func)  # Preserve func's metadata
    def wrapper(*args, **kwargs):
        
        print(f"Calling function: {func.__name__}")
        result = func(*args, **kwargs)
        print(f"Finished calling: {func.__name__}")
        return result
        
    return wrapper

In [34]:

@log_calls
def fetch_data(source):
    """Fetch data from a given source"""
    print(f"Fetching data from {source}")
    return {"data": [1, 2, 3]}

In [35]:
# Call the decorated function
print(fetch_data("Database"))

# Check metadata is preserved
print(fetch_data.__name__)       # Output: fetch_data
print(fetch_data.__doc__)        # Output: Fetch data from a given source

Calling function: fetch_data
Fetching data from Database
Finished calling: fetch_data
{'data': [1, 2, 3]}
fetch_data
Fetch data from a given source


## 4. @contextlib.contextmanager – Custom Context Manager with Parameters

### What is `@contextlib.contextmanager`?

Python's **with** statement is commonly used to manage resources like files and connections, ensuring they are properly opened and closed. Behind the scenes, this is made possible by context managers - objects that set things up and clean them up automatically.

Sometimes, we want to build our own custom context managers. This is where the **@contextmanager** decorator from the **contextlib** module becomes useful. It allows us to turn a generator function into a context manager, using **yield** to mark where the setup ends and cleanup begins. 

#### Business Use Case: Measuring and Logging Time Taken for Different Tasks with Custom Messages

You want to measure task duration and log customized messages before and after the task runs.


In [36]:
import time
from contextlib import contextmanager

In [39]:
# Timer context manager with customizable start and end messages

@contextmanager
def timer(task_name: str, verbose: bool = True):
    
    if verbose:
        print(f"[START] Task: {task_name}")
        
    start_time = time.time()
    
    yield  # Control goes to the 'with' block here
    
    end_time = time.time()
    
    if verbose:
        print(f"[END] Task: {task_name} took {end_time - start_time:.2f} seconds")

In [40]:
# Usage example

def process_data():
    print("Processing data...")
    time.sleep(2)

def save_results():
    print("Saving results...")
    time.sleep(1)

In [42]:
# Measure processing with verbose output

with timer("Data Processing", verbose=True):
    process_data()


# Measure saving without verbose output

with timer("Save Results", verbose=False):
    save_results()

print("All tasks completed.")

[START] Task: Data Processing
Processing data...
[END] Task: Data Processing took 2.00 seconds
Saving results...
All tasks completed.


In [46]:
# Another example
from contextlib import contextmanager
import time

@contextmanager
def log_timer(task_name):
    start = time.time()
    print(f'[{task_name}] started...')
    yield
    end = time.time()
    print(f'[{task_name}] completed in {end-start:.2f} seconds')

with log_timer('process_data'):
    time.sleep(2)

with log_timer('save_results'):
    time.sleep(1)

[process_data] started...
[process_data] completed in 2.01 seconds
[save_results] started...
[save_results] completed in 1.00 seconds


In the above code:
- **log_time** wraps any block of code
- **yield** separates the setup and teardown logic
- Using **with**, you can log timing details around different tasks with minimal code repetition

This pattern is especially useful in measuring performance, controlling resources, and adding temporary behaviours (like verbose output) during specific tasks.


The **@contextmanager** decorator lets you build clean and reusable context-aware blocks for setup and teardown processes. It’s a powerful technique for writing well-structured and resource-safe code.

## 5. @atexit.register – Execute Functions on Program Exit

### What is `@atexit.register`?
In some long-running scripts or data pipelines, there may be a need to perform final clean-up actions like closing files, saving logs, or releasing resources, right before the program exits. The **@atexit.register** decorator allows you to define functions that should run automatically when the program is about to terminate normally.


This is particularly useful in ensuring that no final step is missed, even if it’s not explicitly called at the end of your code.
This decorator registers a function to be executed **automatically when the Python program is about to exit**.  
Useful for cleanup tasks like saving logs, closing resources, or summarizing results.


#### Business Use Case: Saving Application State or Logs Before Shutdown

Imagine an analytics tool that must save user session logs or summaries before the app closes.


In [52]:
!python S1-seg3-1.py

Program is running...
Saving logs before program exit...


In [55]:
# Another example
import atexit

@atexit.register
def goodbye():
    print('Program is exiting.Cleaning up..')

print('Main program is running')

Main program is running


In [56]:
goodbye()

Program is exiting.Cleaning up..


When you run this script, the **goodbye()** function is automatically called when the program finishes, even if it ends without explicitly reaching a final line.

This is particularly useful when:
- Logging the exit status of a script
- Saving unsaved work
- Releasing resources like the database or API connections

It’s important to note that **@atexit.register** only works during a normal shutdown, not in the case of forced exits or crashes. That’s where signal handling (like catching keyboard interrupts) becomes important.

## 6. @signal.signal – Custom Handler for System Signals

### What is `@signal.signal`?
In long-running processes such as training models, data ingestion, or automated monitoring, it’s important to handle unexpected interruptions gracefully. This is where the **signal** module becomes useful.

The **@signal.signal** function allows your program to listen for operating system signals, such as a **KeyboardInterrupt** from pressing Ctrl+C. By catching and responding to these signals, your script can clean up resources, save progress, or print a clear message instead of crashing abruptly.

Allows your Python program to **catch system signals** like `SIGINT` (Ctrl+C) and run a custom handler function.  
Useful for cleaning up or saving data when a user interrupts the program.

#### Business Use Case: Graceful Shutdown of a Data Processing Script

You want your data processing script to save progress or release resources when the user presses Ctrl+C.


In [58]:
!python S1-seg3-2.py

Processing... Press Ctrl+C to stop.
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
Working...
^C

Interrupt received! Cleaning up before exit...


In [60]:
# Here’s a basic example using signal.SIGINT, which is triggered by Ctrl+C

import signal
import time
import sys

def handler(signnum,frame):
    print('\n Interrupt recieved. Shutting down safely')
    sys.exit(0)

signal.signal(signal.SIGINT,handler)

print('Press CTRL+C to interrupt')
while True:
    time.sleep(1)

Press CTRL+C to interrupt

 Interrupt recieved. Shutting down safely


SystemExit: 0

> Handling signals allows for better error handling, logging, and recovery, especially in automated or long-running tasks.

### Common Signals in Python’s `signal` Module

You can experiment with handling these signals in your programs:

| Signal Name       | Description                          | Typical Use Case                     |
|-------------------|------------------------------------|------------------------------------|
| `SIGINT`          | Interrupt from keyboard (Ctrl+C)   | Graceful shutdown                   |
| `SIGTERM`         | Termination signal                 | Controlled termination             |
| `SIGHUP`          | Hangup detected on controlling terminal or death of controlling process | Reload config without stopping    |
| `SIGQUIT`         | Quit from keyboard (Ctrl+\\)         | Generate core dump for debugging   |
| `SIGALRM`         | Alarm clock signal                  | Timers and timeouts                |
| `SIGUSR1` & `SIGUSR2` | User-defined signals             | Custom application events          |

---

### Try this:

- Write handlers for `SIGTERM` or `SIGALRM`.
- Use `signal.pause()` to wait for a signal.
- See how your program reacts on receiving these signals.

---

### Note:

Signal behavior can vary across operating systems; some signals may not be available on Windows.
